In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, LassoCV, Ridge, RidgeCV

### Preparing Datasets

In [4]:
filepath = "C:\\Users\\najiy\\OneDrive\\Desktop\\BerkeleyStuff\\Semesters\\Fall2020\\Stat151A\\Assignments\\Project\\hot-100\\"

#Billboard Hot 100 data
hot_weekly = pd.read_csv(filepath+"HotWeekly.csv")

#Spotify audio features data
hot_features = pd.read_excel(filepath+"Features.xlsx") 

In [5]:
#convert WeekID's to datetime which will be handy for cleaning
hot_weekly['WeekID'] = pd.to_datetime(hot_weekly['WeekID'])

In [6]:
#hot_weekly has lots of repetition for many songs, we only really are interested in aggregate information. 
#we'll do a left join keeping final weekly appearances and matching it to it's appearance in hot_features.

In [8]:
#Add entry month and year columns
hot_weekly['entry_month'] = hot_weekly['WeekID'].dt.month_name()
hot_weekly['year'] = hot_weekly['WeekID'].dt.year

In [9]:
hot_weekly.sort_values(by = 'WeekID', inplace=True)

In [10]:
#Some songs reappear in the chart years after first entries
#We are interested in aggregate information for songs first time entry alone

#
hot_weekly = hot_weekly.merge(hot_weekly.groupby('SongID')[['year']].first().reset_index(),
                 how = 'left',
                 left_on = 'SongID',
                 right_on = 'SongID')

hot_weekly = hot_weekly[~(abs(hot_weekly['year_x'] - hot_weekly['year_y']) > 4)]

In [12]:
#final weekly data with relevant columns
hot_weekly_final = hot_weekly.sort_values(
    by='WeekID',
    ascending = False).groupby('SongID')[['Peak Position',
                                          'Week Position',
                                          'Weeks on Chart',
                                          'entry_month',
                                          'year_y',
                                          'WeekID']].agg({
    'Week Position': lambda x: x.iloc[-1],
    'Peak Position': lambda x: x.iloc[0],
    'Weeks on Chart': lambda x: x.iloc[0],
    'entry_month': lambda x: x.iloc[-1], 
    'year_y' : lambda x: x.iloc[0],
    'WeekID' : lambda x: x.iloc[-1]}).reset_index()

In [13]:
(hot_features['SongID'].value_counts().max() > 1, 
 hot_weekly_final['SongID'].value_counts().max() > 1)

(True, False)

In [14]:
#Some songs in features datasets also appear twice so lets clean that up as well
hot_features_clean = hot_features.groupby('SongID').first().reset_index()

In [15]:
(hot_features_clean['SongID'].value_counts().max() > 1, 
 hot_weekly_final['SongID'].value_counts().max() > 1)

(False, False)

In [16]:
hot_weekly_final.sort_values(by = ['year_y','WeekID'], inplace=True)

#Additional field for aggregate songs count at the time of a songs release in the given year.
hot_weekly_final['yr_chart_ct'] =hot_weekly_final.groupby('year_y').cumcount()+1

In [17]:
#finally we have clean datasets so we can go ahead and join
hot_merged = hot_weekly_final.merge(
    hot_features_clean, 
    how = "left", 
    left_on = "SongID", 
    right_on = "SongID")

In [18]:
#Many records have a bunch of missing values, and basically that's just what it is, missing information
#We need every information to do our analysis here and since removing songs doesnt really affect our goals we'll just
#Drop all records with missing values
#hot_merged.dropna(inplace=True)

In [19]:
#some of the names are kinda long, also inconsistent use of capitlizations so lets clean that up too
hot_merged.rename({'SongID' : 'songid',
                   'WeekID' : 'entry_weekid',
                   'Week Position': 'entry_position',
                   'Week Position2': 'avg_pos',
                   'Peak Position' : 'peak',
                   'Weeks on Chart': 'weeks',
                   'Performer' : 'performer',
                   'Song':'song',
                   'spotify_genre' : 'st_genre',
                   'spotify_track_album':'st_album',
                   'spotify_track_explicit': 'st_explicit',
                   'spotify_track_duration_ms':'st_duration_ms',
                   'spotify_track_popularity':'st_popularity' }, axis = 1, inplace = True)

### Further Data Cleaning/Mod

In [204]:
hot_merged.describe()

,entry_position,peak,weeks,year_y,yr_chart_ct,st_explicit,st_duration_ms,st_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,28474.000000,28474.000000,28474.00000,28474.000000,28474.000000,23634.000000,2.363400e+04,23634.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000,23572.000000
mean,81.012748,47.189015,11.23748,1985.276709,248.060441,0.108953,2.214737e+05,40.360921,0.597956,0.618061,5.242491,-8.723276,0.729722,0.071608,0.296239,0.033375,0.192719,0.605555,120.203157,3.930553
std,18.338973,30.970541,8.13157,18.659866,163.086675,0.311587,6.819471e+04,22.095835,0.152898,0.200157,3.560444,3.613220,0.444113,0.080373,0.283015,0.138096,0.159914,0.238231,27.988539,0.320639
min,1.000000,1.000000,1.00000,1958.000000,1.000000,0.000000,2.968800e+04,0.000000,0.000000,0.000581,0.000000,-28.030000,0.000000,0.000000,0.000003,0.000000,0.009670,0.000000,0.000000,0.000000
25%,75.000000,18.000000,5.00000,1969.000000,115.000000,0.000000,1.754260e+05,22.000000,0.497000,0.473000,2.000000,-11.115250,0.000000,0.032000,0.046775,0.000000,0.090500,0.421000,99.106250,4.000000
50%,86.000000,47.000000,10.00000,1982.000000,230.000000,0.000000,2.157595e+05,42.000000,0.606000,0.634000,5.000000,-8.300000,1.000000,0.040900,0.197000,0.000005,0.131000,0.627000,118.779500,4.000000
75%,94.000000,75.000000,16.00000,2002.000000,351.000000,0.000000,2.543060e+05,58.000000,0.706000,0.779000,8.000000,-5.901000,1.000000,0.066500,0.514000,0.000510,0.250000,0.805000,136.102500,4.000000
max,100.000000,100.000000,87.00000,2019.000000,740.000000,1.000000,3.079157e+06,100.000000,0.988000,0.997000,11.000000,2.291000,1.000000,0.951000,0.991000,0.982000,0.999000,0.991000,241.009000,5.000000


In [205]:
pattern = r"[^a-zA-Z]"
hot_merged['st_genre2'] = hot_merged['st_genre'].str.lower().str.replace(pattern, "")
hot_merged['performer'] = hot_merged['performer'].str.lower().str.replace(pattern, "")

In [206]:
pattern = r"[^a-zA-Z,]"
hot_merged['st_genre'] = hot_merged['st_genre'].str.lower().str.replace(
    pattern, "").str.split(",")

In [207]:
#Add a field for select song genres.
gnr_dict = {'pop':r'pop',
 'country':r"country|folk",
 'hiphop':r'rap|hiphop|trap',
 'r&b':r'rb|rhythmandblues',
 'rock':r'rock|rockandroll',
 'dance':'disco|dance'}

for key in gnr_dict:
    hot_merged[key] = hot_merged['st_genre2'].str.contains(gnr_dict[key])*1

In [208]:
def genre_match(dflist, gdict):
    if type(dflist) is list:
        for genre in dflist:
            for key in gdict:
                if genre in gdict[key]:
                    return key
    return 'other'

#Different genre field approach
gnr_dict2 = {'pop':['pop', 'dancepop', 'kpop'],
 'country':['country', 'folk'],
 'hiphop':['rap','hiphop','trap'],
 'r&b':['rb' 'rhythmandblues'],
 'rock':['rock', 'rockandroll'],
 'dance':['disco', 'dance']}

hot_merged['genre'] = hot_merged['st_genre'].apply(lambda x: genre_match(x, gnr_dict2))

In [209]:
#Add a field for weather a song had a featured artist
pattern = r"(featuring\w+)"
hot_merged['feat_artist'] = hot_merged['performer'].str.extract(
    pattern).replace({np.nan:'None'})[0]
hot_merged['performer'] = hot_merged['performer'].str.replace(pattern,"")

In [210]:
hot_merged['feat_artist'] = hot_merged['feat_artist'].str.replace(r"featuring", "")

In [211]:
#Add a field for count of previous charted songs by an artist
hot_merged['art_prev_100_ct'] = hot_merged.groupby('performer').cumcount()

In [212]:
hot_merged['has_feat'] = ~(hot_merged['feat_artist'] == 'None')+0

In [214]:
#Export current dataframe
hot_merged.drop(['spotify_track_id',
                 'song',
                 'spotify_track_preview_url',
                 'st_album',
                 'st_genre',
                 'st_genre2'], axis=1).to_csv(filepath+"hot_merged_full.csv")

### Regression Practice

In [171]:
for col in hot_merged.columns.values:
    if type(hot_merged[col][0]) == np.float64:
        hot_merged[col] = hot_merged[col].round(decimals=3)

In [26]:
def rmse(y_obs, y_pred):
    return np.sqrt(np.mean((y_obs-y_pred)**2))

def oh_encode_no_redundant(data):
    """
    Return the one-hot encoded dataframe of our input data, removing redundancies.
    
    Parameters
    -----------
    data: a dataframe that may include non-numerical features
    
    Returns
    -----------
    A one-hot encoded dataframe that only contains numeric features without any redundancies.
    
    """
    encoded_data = pd.get_dummies(data)
    indices = np.where(np.corrcoef(encoded_data.T)[0] > 0.6)[0]
    if indices.size > 1:
        redundant_cols = encoded_data.columns[indices[1:]]
        return encoded_data.drop(redundant_cols, axis=1)
    else:
        return encoded_data
    
def guided_feature_eng(data, preprocess=None):
    data = oh_encode_no_redundant(data.copy())
    if preprocess is not None:
        return preprocess.fit_transform(data)
    return data

In [105]:
features_id = ['year','entry_position', 'st_duration_ms', 'danceability', 'energy',
               'key','loudness', 'mode', 'speechiness','acousticness', 'instrumentalness',
               'liveness', 'valence', 'tempo','st_explicit', 'has_feat_artist', 'genre']

        
#seperate into training and testing data   
tr_response = hot_merged.query("year < 2010").dropna()['weeks']
tr_features = hot_merged.query("year < 2010").dropna().set_index('songid')[features_id]

te_response = hot_merged.query("year >= 2010").dropna()['weeks']
te_features = hot_merged.query("year >= 2010").dropna().set_index('songid')[features_id]

In [106]:
from sklearn.metrics import SCORERS

In [107]:
#first model fit
lin_model = LinearRegression()
tr_feat_basic = guided_feature_eng(tr_features)


lin_model.fit(tr_feat_basic, tr_response);
lin_model.score(tr_feat_basic, tr_response)

0.23389646661308505

In [108]:
#training set performance test with cross validation
cross_validate(LinearRegression(), tr_feat_basic, tr_response, scoring='neg_root_mean_squared_error')

{'fit_time': array([0.02500582, 0.00700164, 0.00900221, 0.00700188, 0.0070014 ]),
 'score_time': array([0.00100088, 0.00100017, 0.00099993, 0.00100017, 0.        ]),
 'test_score': array([-6.56650328, -6.74207243, -6.31628242, -6.35048426, -6.60436557])}

### Exploring Improvements

In [109]:
from sklearn.preprocessing import PolynomialFeatures

In [110]:
inter_poly = PolynomialFeatures(interaction_only=True)
poly_pw = PolynomialFeatures(degree=2)
lasso_model = Lasso()
ridge_model = Ridge()

In [111]:
tr_feat_inter = guided_feature_eng(tr_features, inter_poly)
tr_feat_quad = guided_feature_eng(tr_features, poly_pw)

In [112]:
#standard and regularized models training data fit and error check
lin_model.fit(tr_feat_basic, tr_response)
lasso_model.fit(tr_feat_basic, tr_response)
ridge_model.fit(tr_feat_basic, tr_response)

(rmse(tr_response, lasso_model.predict(tr_feat_basic)),
 rmse(tr_response, ridge_model.predict(tr_feat_basic)))

(6.556574733277452, 6.501736824045065)

In [113]:
lin_model.fit(tr_feat_quad, tr_response);
(rmse(tr_response, lin_model.predict(tr_feat_quad)), lin_model.score(tr_feat_quad, tr_response))

(6.3585297351207375, 0.26727235980666697)

In [115]:
lin_model.fit(tr_feat_inter, tr_response);
(rmse(tr_response, lin_model.predict(tr_feat_inter)) , lin_model.score(tr_feat_inter, tr_response))

(6.399816445012737, 0.2577260874893883)

In [117]:
#check performance of features w/interaction using cross validation
cross_validate(LinearRegression(),
               tr_feat_inter,
               tr_response,
               scoring='explained_variance')

{'fit_time': array([0.11953259, 0.18204093, 0.08101797, 0.17103887, 0.08001828]),
 'score_time': array([0.00200033, 0.00100255, 0.00200248, 0.00200295, 0.00200009]),
 'test_score': array([0.20741118, 0.21399939, 0.23640651, 0.18723357, 0.24502156])}

In [103]:
#check performance of quadratic features using cross validation
cross_validate(LinearRegression(),
               tr_feat_quad,
               tr_response,
               scoring='neg_root_mean_squared_error')

{'fit_time': array([0.15103412, 0.1330297 , 0.11202526, 0.15803552, 0.11202502]),
 'score_time': array([0.00200319, 0.00200057, 0.00200033, 0.00200057, 0.00100017]),
 'test_score': array([-3.90638835, -3.89530114, -3.6192317 , -3.58797865, -3.90352306])}

### Logistic Regression V2

In [36]:
#Using initial hot_merged dataset, will add a rank_level variable to be used as a 10 class categorical variable
rank_dict = {}
rank_dict["Top 10"] = np.arange(start= 1, stop=11, step=1)
rank_dict["Not Top 10"] = np.arange(start = 11, stop=101, step= 1)

def rank_match(rank):
    for key in rank_dict.keys():
        if rank in rank_dict[key]:
            return key
        
hot_merged['rank_level'] = hot_merged['peak'].apply(lambda x: rank_match(x))

In [37]:
balanced_dat = pd.concat([hot_merged.query("peak > 10").sample(n=4000), 
           hot_merged.query("peak <= 10").sample(n=4000)])



In [38]:
features_id = ['entry_position', 'artist_prev_hot100_songs', 'st_duration_ms', 'danceability', 'energy',
               'key','loudness', 'mode', 'speechiness','acousticness', 'instrumentalness',
               'liveness', 'valence', 'tempo','st_explicit', 'has_feat_artist', 
               'pop','country','rock','hiphop','r&b','dance']

        
#seperate into training and testing data   
tr_response = balanced_dat.query("year < 2010").dropna()['rank_level']
tr_features = balanced_dat.query("year < 2010").dropna().set_index('songid')[features_id]

te_response = balanced_dat.query("year >= 2010").dropna()['rank_level']
te_features = balanced_dat.query("year >= 2010").dropna().set_index('songid')[features_id]

In [39]:
tr_features['st_duration_ms'] = (tr_features['st_duration_ms']- np.mean(tr_features['st_duration_ms']))/np.std(tr_features['st_duration_ms'] )

In [40]:
log_model = LogisticRegression(fit_intercept = True, solver = 'lbfgs', max_iter=1000)

In [41]:
log_model.fit(guided_feature_eng(tr_features), tr_response)
print(classification_report(tr_response, log_model.predict(guided_feature_eng(tr_features))))

              precision    recall  f1-score   support

  Not Top 10       0.63      0.66      0.65      1600
      Top 10       0.69      0.67      0.68      1856

    accuracy                           0.67      3456
   macro avg       0.66      0.67      0.66      3456
weighted avg       0.67      0.67      0.67      3456



C:\Users\najiy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
